In [1]:
import pandas as pd
import sqlite3

In [2]:
providers = pd.read_csv("providers_data.csv")
receivers = pd.read_csv("receivers_data.csv")
food_listings = pd.read_csv("food_listings_data.csv")
claims = pd.read_csv("claims_data.csv")

In [3]:
connection = sqlite3.connect("food_waste_management.db")
cursor = connection.cursor()

In [9]:
cursor.execute('''CREATE TABLE IF NOT EXISTS providers( 
    provider_id integer primary key,
    name varchar(100),
    type varchar(50),
    address varchar(250),
    city varchar(50),
    contact varchar(50)
    )
''')

In [10]:
for index, row in providers.iterrows():
    cursor.execute('''
    INSERT INTO providers (provider_id,name,type,address,city,contact)
    VALUES (?, ?, ?, ?, ?, ?)
    ''', row
    )

connection.commit()

C:\Users\monoh\AppData\Local\Temp\ipykernel_11176\1074358683.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cursor.execute('''


In [11]:
cursor.execute('''CREATE TABLE IF NOT EXISTS receivers( 
    receiver_id integer primary key,
    name varchar(100),
    type varchar(50),
    city varchar(50),
    contact varchar(50)
    )
''')

In [12]:
for index, row in receivers.iterrows():
    cursor.execute('''
    INSERT INTO receivers (receiver_id,name,type,city,contact)
    VALUES (?, ?, ?, ?, ?)
    ''', row
    )

connection.commit()

C:\Users\monoh\AppData\Local\Temp\ipykernel_11176\2171948081.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cursor.execute('''


In [13]:
cursor.execute('''CREATE TABLE IF NOT EXISTS food_listings( 
    food_id integer primary key,
    food_Name varchar(100),
    quantity interger,
    expiry_date date,
    provider_id int,
    provider_type varchar(100),
    location varchar(100),
    food_type varchar(100),
    meal_type varchar(50)
    )
''')

In [14]:
for index, row in food_listings.iterrows():
    cursor.execute('''
    INSERT INTO food_listings (food_id,food_name,quantity,expiry_date,provider_id,provider_type,location,food_type,meal_type)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', row
    )

connection.commit()

C:\Users\monoh\AppData\Local\Temp\ipykernel_11176\2192409620.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cursor.execute('''


In [15]:
cursor.execute('''CREATE TABLE IF NOT EXISTS claims( 
    claim_id integer primary key,
    food_id integer,
    receiver_id integer,
    status varchar(50),
    time_stamp datetime
    )
''')

In [16]:
for index, row in claims.iterrows():
    cursor.execute('''
    INSERT INTO claims (claim_id,food_id,receiver_id,status,time_stamp)
    VALUES (?, ?, ?, ?, ?)
    ''', row
    )

connection.commit()

C:\Users\monoh\AppData\Local\Temp\ipykernel_11176\2923405635.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cursor.execute('''


In [12]:
#query 1.How many food providers and receivers are there in each city?

query_1_1 = ('''
            select city, count(provider_id) from providers group by city;
 ''')

query_1_2 = ('''
            select city, count(receiver_id) from receivers group by city;
 ''')

cursor.execute(query_1_1)
result_1_1 = cursor.fetchall()

cursor.execute(query_1_2)
result_1_2 = cursor.fetchall()

df_1_1 = pd.DataFrame(result_1_1,columns=["city","no_of_food_providers"])
df_1_2 = pd.DataFrame(result_1_2,columns=["city","no_of_food_receivers"])


combined_df = pd.concat([df_1_1, df_1_2], axis=1, ignore_index=False)
display(combined_df)

,city,no_of_food_providers,city,no_of_food_receivers
0,Adambury,1.0,Aaronshire,1
1,Adamsview,1.0,Adamland,1
2,Adamsville,1.0,Aguilarbury,1
3,Aguirreville,1.0,Aguilarstad,1
4,Alexanderchester,1.0,Alexanderbury,1
...,...,...,...,...
961,Zimmermanton,1.0,Woodardview,1
962,Zimmermanville,1.0,Woodsfurt,1
963,NaN,NaN,Wrightland,1
964,NaN,NaN,Wyattton,1


In [13]:
#query 2.Which type of food provider (restaurant, grocery store, etc.) contributes the most food?

query_2 = ('''
            select provider_type,sum(quantity) as contributed_food
            from food_listings
            group by provider_type
            order by sum(quantity) desc
            limit 1;
 ''')

cursor.execute(query_2)
result_2 = cursor.fetchall()

df_2 = pd.DataFrame(result_2,columns=["provider_type","contributed_food"])
df_2

,provider_type,contributed_food
0,Restaurant,6923


In [14]:
#query 3.What is the contact information of food providers in a specific city?***

query_3 = ('''
            SELECT name,contact FROM providers
            group by city;
 ''')

cursor.execute(query_3)
result_3 = cursor.fetchall()

df_3 = pd.DataFrame(result_3,columns=["provider_name","contact"])
df_3

,provider_name,contact
0,Ibarra LLC,6703380260
1,Lozano-Miller,001-281-026-8022
2,Davis Ltd,(112)122-3591x558
3,Rowe-Chen,8228891240
4,Galloway-Henderson,001-867-928-0212x3211
...,...,...
958,Williams PLC,(730)287-3289
959,Frederick LLC,784.860.0232x137
960,"Obrien, King and Robinson",+1-513-264-8229x419
961,Larson and Sons,152.836.4864


In [15]:
#query 4.Which receivers have claimed the most food?
query_4 = ('''
            select receivers.name, sum(food_listings.quantity) as food_claimed
            from receivers
            inner join claims on receivers.receiver_id = claims.receiver_id
            inner join food_listings on food_listings.food_id = claims.food_id
            group by receivers.receiver_id
            order by food_claimed desc
            limit 10;
 ''')

cursor.execute(query_4)
result_4 = cursor.fetchall()

df_4 = pd.DataFrame(result_4,columns=["receiver_name","food_claimed"])
df_4

,receiver_name,food_claimed
0,Matthew Webb,191
1,Donald Caldwell,174
2,Anthony Garcia,162
3,Scott Hunter,157
4,Jennifer Nelson,139
5,Chelsea Powell,134
6,Justin Powell,126
7,Kristina Simpson,124
8,Jose Vance,123
9,Donna Williams,122


In [16]:
#query 5.What is the total quantity of food available from all providers?

query_5 = ('''
            select sum(quantity) as total_quantity_of_food_available
            from food_listings
            inner join claims on claims.Food_id = food_listings.Food_id
            where claims.status != 'completed';
 ''')

cursor.execute(query_5)
result_5 = cursor.fetchall()

df_5 = pd.DataFrame(result_5,columns=["total_quantity_of_food_available"])
df_5

,total_quantity_of_food_available
0,25959


In [17]:
#query 6.Which city has the highest number of food listings?

query_6 = ('''
            select location,sum(quantity) as food_listed 
            from food_listings
            group by location
            order by food_listed desc
            limit 1;
 ''')

cursor.execute(query_6)
result_6 = cursor.fetchall()

df_6 = pd.DataFrame(result_6,columns=["location","food_listed"])
df_6

,location,food_listed
0,South Kathryn,179


In [18]:
#query 7.What are the most commonly available food types?

query_7 = ('''
            select distinct(Food_type) as food_types
            from food_listings
 ''')

cursor.execute(query_7)
result_7 = cursor.fetchall()

df_7 = pd.DataFrame(result_7,columns=["food_types"])
df_7

,food_types
0,Non-Vegetarian
1,Vegan
2,Vegetarian


In [21]:
#query  9.How many food claims have been made for each food item?

query_9 = ('''
            select food_name,count(claims.food_id) as no_of_claims
            from food_listings
            inner join claims on claims.food_id = food_listings.food_id
            group by food_name;
 ''')

cursor.execute(query_9)
result_9 = cursor.fetchall()

df_9 = pd.DataFrame(result_9,columns=["food_name","no_of_claims"])
df_9

,food_name,no_of_claims
0,Bread,94
1,Chicken,102
2,Dairy,110
3,Fish,108
4,Fruits,71
5,Pasta,87
6,Rice,122
7,Salad,106
8,Soup,114
9,Vegetables,86


In [22]:
#query  10.Which provider has had the highest number of successful food claims?
query_10 = ('''
            select providers.name, count(claims.status) as sucessful_food_claims
            from providers
            inner join food_listings on providers.provider_id = food_listings.provider_id
            inner join claims on claims.food_id = food_listings.food_id
            where claims.status = 'Completed'
            group by providers.provider_id
            order by sucessful_food_claims desc
            limit 1;
 ''')

cursor.execute(query_10)
result_10 = cursor.fetchall()

df_10 = pd.DataFrame(result_10,columns=["provider_name","Sucessful_food_claims"])
df_10

,provider_name,Sucessful_food_claims
0,Barry Group,5


In [23]:
#query  12.What percentage of food claims are completed vs. pending vs. canceled?

query_12 = ('''
            select (select (count(status)*100)/(select count(*) from claims)
            from claims 
            where status = "Completed"),(select (count(status)*100)/(select count(*) from claims)
            from claims 
            where status = "Pending"),(select (count(status)*100)/(select count(*) from claims)
            from claims 
            where status = "Cancelled") from claims
            limit 1;
 ''')

cursor.execute(query_12)
result_12 = cursor.fetchall()

df_12 = pd.DataFrame(result_12,columns=["completed","pending","cancelled"])
df_12

,completed,pending,cancelled
0,33,32,33


In [24]:
#query  13.What is the average quantity of food claimed per receiver?

query_13 = ('''
                select receivers.name,avg(food_listings.quantity) as average_quantity_of_food_claimed_per_receiver
                from food_listings
                inner join claims on food_listings.food_id = claims.food_id
                inner join receivers on claims.receiver_id = receivers.receiver_id
                group by receivers.receiver_id;
 ''')

cursor.execute(query_13)
result_13 = cursor.fetchall()

df_13 = pd.DataFrame(result_13,columns=["receiver_name","average_quantity_of_food_claimed_per_receiver"])
df_13

,receiver_name,average_quantity_of_food_claimed_per_receiver
0,Donald Gomez,35.5
1,Laurie Ramos,47.0
2,Ashley Mckee,26.0
3,Erika Rose,30.5
4,Mandy Sutton PhD,15.0
...,...,...
619,Brittany Taylor,35.0
620,Matthew Curtis,24.5
621,Amanda Cain,25.5
622,Cheyenne Ramsey,42.0


In [25]:
#query  14.Which meal type (breakfast, lunch, dinner, snacks) is claimed the most?

query_14 = ('''
                select food_listings.meal_type as meal_type,count(claims.claim_id)
                from food_listings
                inner join claims on claims.food_id = food_listings.food_id 
                where claims.status = 'Completed'
                group by food_listings.meal_type
                order by count(claims.claim_id)
                limit 1;
 ''')

cursor.execute(query_14)
result_14 = cursor.fetchall()

df_14 = pd.DataFrame(result_14,columns=["meal_type","no_of_claims"])
df_14

,meal_type,no_of_claims
0,Snacks,74


In [26]:
#query  15.What is the total quantity of food donated by each provider?

query_15 = ('''
                select providers.name,sum(food_listings.quantity)
                from providers
                inner join food_listings on food_listings.provider_id = providers.provider_id
                group by providers.name; 
 ''')

cursor.execute(query_15)
result_15 = cursor.fetchall()

df_15 = pd.DataFrame(result_15,columns=["provider_name","quantity_donated"])
df_15

,provider_name,quantity_donated
0,"Abbott, Brooks and Moreno",47
1,Abbott-Robinson,44
2,Adams-Meyer,28
3,Adams-Young,16
4,Aguilar Group,66
...,...,...
623,"Young, Townsend and Mccarthy",87
624,Young-Luna,35
625,Young-Wu,2
626,Yu-Rodriguez,14


In [27]:
#query  16.which provider type have provided the least food?

query_16 = ('''
                select providers.type, sum(food_listings.quantity)
                from providers
                inner join food_listings on food_listings.provider_id = providers.provider_id
                group by providers.type
                order by sum(food_listings.quantity) asc
                limit 1 ;   
 ''')

cursor.execute(query_16)
result_16 = cursor.fetchall()

df_16 = pd.DataFrame(result_16,columns=["provider_type","quantity"])
df_16

,provider_type,quantity
0,Grocery Store,6059


In [28]:
#query  17.which city have more providers ?

query_17 = ('''
                select city,count(provider_id)
                from providers
                group by city
                order by count(provider_id) desc
                limit 1;               
 ''')

cursor.execute(query_17)
result_17 = cursor.fetchall()

df_17 = pd.DataFrame(result_17,columns=["city","no_of_providers"])
df_17

,city,no_of_providers
0,South Christopherborough,3


In [29]:
#query  18.which type of receivers claimed more food

query_18 = ('''
                select receivers.type ,sum(quantity)
                from receivers
                inner join claims on claims.receiver_id = receivers.receiver_id
                inner join food_listings on food_listings.food_id = claims.food_id
                group by receivers.type
                order by sum(quantity) desc
                limit 1;     
 ''')

cursor.execute(query_18)
result_18 = cursor.fetchall()

df_18 = pd.DataFrame(result_18,columns=["receiver_type","food_claimed"])
df_18

,receiver_type,food_claimed
0,Charity,7168


In [35]:
#query  19.Top 5 city has low number of food listings

query_19 = ('''
                select location, sum(quantity) as quantity_listed
                from food_listings
                group by location
                order by quantity_listed asc
                limit 5;                  
 ''')

cursor.execute(query_19)
result_19 = cursor.fetchall()

df_19 = pd.DataFrame(result_19,columns=["city_name","food_listed"])
df_19

,city_name,food_listed
0,Lake Anthonyport,1
1,New Hollyfurt,1
2,New Monicaside,1
3,Velazquezview,1
4,Andersonville,2


In [31]:
#query  20.top 10 provider has provided vegan foodtype

query_20 = ('''
                select providers.name as provider_name
                from providers
                join food_listings on food_listings.provider_id = providers.provider_id
                where food_listings.food_type = 'Vegan'
                order by food_listings.quantity desc
                limit 10;               
 ''')

cursor.execute(query_20)
result_20 = cursor.fetchall()

df_20 = pd.DataFrame(result_20,columns=["provider_name"])
df_20

,provider_name
0,"Lucas, Bush and Miller"
1,Rios Group
2,Cox LLC
3,Schmidt Ltd
4,"Horn, Johnson and Moore"
5,Torres-Dunlap
6,"Rodriguez, Walker and Singleton"
7,Woodward-Jensen
8,"Hodge, Lee and Washington"
9,Glass Inc


In [32]:
#query  21.what is the average quantity of food donated by each provider

query_21 = ('''
                select providers.name as provider_name,avg(food_listings.quantity) as average_quantity_of_food
                from providers
                inner join food_listings on providers.provider_id = food_listings.provider_id
                group by providers.provider_id               
 ''')

cursor.execute(query_21)
result_21 = cursor.fetchall()

df_21 = pd.DataFrame(result_21,columns=["provider_name","average_quantity_of_food"])
df_21

,provider_name,average_quantity_of_food
0,Gonzales-Cochran,39.0
1,"Nielsen, Johnson and Fuller",26.5
2,"Clark, Prince and Williams",32.0
3,Coleman-Farley,46.0
4,Ortiz-Lee,38.0
...,...,...
632,Moore Group,26.5
633,Garza-Williams,47.0
634,Novak Group,7.0
635,Moody Ltd,28.5


In [33]:
#query  22.list top 5 provider

query_22 = ('''
                select providers.Name as provider_name, sum(food_listings.quantity) as food_contributed
                from providers
                inner join food_listings on providers.provider_id = food_listings.provider_id
                group by food_listings.provider_id
                order by food_contributed desc
                limit 5;
                
 ''')

cursor.execute(query_22)
result_22 = cursor.fetchall()

df_22 = pd.DataFrame(result_22,columns=["provider_name","Food_contributed"])
df_22

,provider_name,Food_contributed
0,Barry Group,179
1,"Evans, Wright and Mitchell",158
2,Smith Group,150
3,Nelson LLC,142
4,Ruiz-Oneal,140
